<a href="https://colab.research.google.com/github/stkisengese/titanic-survial-prediction-ml/blob/main/notebook/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Titanic Machine Learining - Iteration 1: Naive Baseline

In [1]:
import pandas as pd
import numpy as np

Predict all passagers die(survival = 0)

In [2]:
# Load data
test = pd.read_csv('../data/test.csv')
train = pd.read_csv('../data/train.csv')

In [3]:
# write submission with all passengers predicted as died

submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": 0
})

In [4]:
# Save the submission file
submission.to_csv('../data/gender_submission.csv', index=False)

In [5]:
print(submission.head(10))
print(submission.tail(10))

   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         0
5          897         0
6          898         0
7          899         0
8          900         0
9          901         0
     PassengerId  Survived
408         1300         0
409         1301         0
410         1302         0
411         1303         0
412         1304         0
413         1305         0
414         1306         0
415         1307         0
416         1308         0
417         1309         0
